# **Análisis de Sistemas Térmicos usando Python**

In [45]:
#pip install Coolprop

En este notebook se presenta un ejemplo de cómo usar Python para analizar los procesos internos de un Aire Acondicionado centralizado de un edificio. Se presenta un ejemplo considerando un sistema operando en un edificio y bajo condiciones climáticas de cundía caluroso y húmedo.

Partamos importando las librerías necesarias para trabajar.

In [46]:
import numpy as np
import CoolProp.CoolProp as CP
import matplotlib.pyplot as plt
from CoolProp.HumidAirProp import HAPropsSI
from pint import UnitRegistry

El ejemplo considera que el aire interior se acondiciona a $T_{in} =$ 24ºC y $\phi_{in} =$ 0.45. 

El edificio está sujeto a una carga térmica debido a la ganancia solar, la conducción y la infiltración de aire, así como a las cargas internas asociadas con diversas fuentes. La tasa total de transferencia de calor al edificio es  $\dot{Q}_b =$ 100kW. 

El aire del edificio también está sujeto a la adición de humedad por parte de los ocupantes y otras fuentes. 

La tasa total de adición de humedad al aire del edificio es $\dot{m}_{v,b} =$ 0.0115 kg/s. 

El aire exterior a una temperatura de $T_{out} =$ 30ºC y $\phi_{out} =$ 0.58 se introduce en el sistema a un caudal de $\dot{V}_{out} =$ 5100 cfm. 

Parte del aire de escape extraído del edificio se desvía y se mezcla con el aire exterior. 

La velocidad de flujo volumétrico del flujo de aire de retorno es $\dot{V}_{r} =$ 8200 cfm. 

El flujo de aire mixto pasa a través de una bobina de agua fría donde se enfría y deshumidifica, y luego a través de un serpentín por donde circula vapor, donde se recalienta. 

La presión es atmosférica en todo el sistema.

<img src="Class_16_AC.png" alt="Example" />

In [47]:
ureg = UnitRegistry()
ureg.default_format = '.3f'

a) Determine la temperatura y la humedad relativa del aire que se proporciona al edificio en el estado 5.

In [48]:
#Presión en el sistema
P = (1 *ureg.atm).to('Pa') # Presión atmosférica en Pa

#Edificio
Q_dot_b = (ureg.Quantity(100, 'kW')).to('W') # Carga térmica del edificio
m_dot_b = (ureg.Quantity(0.0115, 'kg/s')).to('kg/s') # Carga de agua en el edificio (carga latente)

## Datos del aire externo  
T_amb = (ureg.Quantity(30, 'degC')).to('K') # Temperatura del aire externo
phi_amb = 0.58  # Humedad relativa del aire externo
V_dot_amb = (ureg.Quantity(5100, 'feet**3/minute')).to('m**3/s') # Caudal de aire externo 

## Datos del aire interno
T_in = (ureg.Quantity(24, 'degC')).to('K') # Temperatura del aire interno   
phi_in = 0.45  # Humedad relativa del aire interno
V_dot_r = (ureg.Quantity(8200, 'feet**3/minute')).to('m**3/s') # Caudal de aire de retorno

In [49]:
T = np.empty(5)*ureg.kelvin
h = np.empty(5)*ureg.joule/ureg.kilogram
v = np.empty(5)*ureg.meter**3/ureg.kilogram
phi,omega = np.empty(5),np.empty(5)

El estado del aire proveniente del exterior (estado 0), está determinado por la temperatura, $T_0 = T_{amb}$, y la humedad relativa, $\phi_0 = \phi_{amb}$. Se calculan la relación de humedad, la entalpía específica del aire seco y el volumen específico del aire seco ($\omega_0$, $h_{0}$ y $v_{0}$).

In [50]:
# Estado 0: Aire de externo

T[0] = T_amb
phi[0] = phi_amb
omega[0] = HAPropsSI('W', 'T', T[0].magnitude, 'R', phi[0], 'P', P.magnitude)
h[0] = HAPropsSI('H', 'T', T[0].magnitude, 'R', phi[0], 'P', P.magnitude)*ureg.joule/ureg.kilogram
v[0] = HAPropsSI('V', 'T', T[0].magnitude, 'R', phi[0], 'P', P.magnitude)*ureg.meter**3/ureg.kilogram

El flujo másico de aire retornado (recirculado) al sistema se determina:

$\dot{m}_{amb} = \frac{\dot{V}_{amb}}{v_{0}}$

In [51]:
m_dot_amb = V_dot_amb/v[0]
m_dot_amb

2.7352366839360274 <Unit('kilogram / second')>

El estado del aire que es retornado desde el efiticio (estado 1) está definido por la temperatura $T_{in} = T_1$ y la humedad relativa $\phi_{in}=\phi_1$. La humedad absoluta, entalpìa y volumen específico en el estado 1 ($h_1$, $v_1$ y $w_1$, respectivamente) son determinados a partir de dichas propiedades.

In [52]:
# Estado 1: Aire de retorno

T[1] = T_in
phi[1] = phi_in
omega[1] = HAPropsSI('W', 'T', T[1].magnitude, 'R', phi[1], 'P', P.magnitude)
h[1] = HAPropsSI('H', 'T', T[1].magnitude, 'R', phi[1], 'P', P.magnitude)*ureg.joule/ureg.kilogram
v[1] = HAPropsSI('V', 'T', T[1].magnitude, 'R', phi[1], 'P', P.magnitude)*ureg.meter**3/ureg.kilogram

El flujo másico de aire retornado (recirculado) al sistema se determina:

$\dot{m}_{r} = \frac{\dot{V}_r}{v_{1}}$

In [53]:
m_dot_r = V_dot_r/v[1]
m_dot_r 

4.537760332898475 <Unit('kilogram / second')>

Un balance de masa de aire seco en estado estacionario en el proceso de mezcla es:
$\dot{m}_{r} + \dot{m}_{amb} = \dot{m}_{a}$

donde $\dot{m}_{a}$ es el flujo másico de aire seco que pasa a través de los serpentines de agua fría y vapor.

Similarmente, aplicando un balance de masa de agua en el proceso de mezcla es posible determinar la relación de humedad del aire que ingresa al serpentín de agua fría. 

$\dot{m}_{r} \omega_1+ \dot{m}_{amb} \omega_0 = \dot{m}_{a} \omega_2$
 

Por otro lado, el balance de energía en estado estacionario en el proceso de mezcla adiabática se expresa como:

​$\dot{m}_{r} h_1+ \dot{m}_{amb} h_0 = \dot{m}_{a} h_2$
 
Desde esta expresión es posible determinar la entalpía específica del aire seco que ingresa al serpentín de agua fría. 

Así, el estado 2 está determinado por $\omega_2$ y $h_2$, luego es posible determinar la temperatura y la humedad relativa del aire en el estado 2

In [57]:
#Mezcla aire de retorne y aire externo
m_dot_a = m_dot_r + m_dot_amb                               #Balance de masa  
omega[2] = (m_dot_r*omega[1] + m_dot_amb*omega[0])/m_dot_a  #Balance de masa de agua
h[2] = (m_dot_r*h[1] + m_dot_amb*h[0])/m_dot_a              #Balance de energía

T[2] =  HAPropsSI('T', 'H', h[2].magnitude, 'W', omega[2], 'P', P.magnitude)*ureg.kelvin #Temperatura de la mezcla
phi[2] = HAPropsSI('R', 'H', h[2].magnitude, 'W', omega[2], 'P', P.magnitude) #Humedad relativa de la mezcla